<a href="https://colab.research.google.com/github/kavyakapoor200/Twitter-Scrapper/blob/main/Twitter_Scrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update
!apt install -y firefox
!apt install -y xvfb
!pip install selenium
!pip install pyvirtualdisplay
!wget https://github.com/mozilla/geckodriver/releases/latest/download/geckodriver-linux64.tar.gz
!tar -xvzf geckodriver-linux64.tar.gz
!chmod +x geckodriver
!mv geckodriver /usr/local/bin/


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [68.9 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,321 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,235 kB]
Hit:13 https://ppa.launchpadco

In [ ]:
import pandas as pd
import re
import time
import random
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from pyvirtualdisplay import Display

# Start virtual display for running Selenium in Colab
display = Display(visible=0, size=(1024, 768))
display.start()

# Step 1: Download CSV file from Google Drive
def download_csv(drive_link, output_file):
    file_id = drive_link.split("/d/")[1].split("/")[0]  # Extract file ID from link
    gdown.download(f"https://drive.google.com/uc?id={file_id}", output_file, quiet=False)

# Function to extract Twitter username from URL
def extract_username(url):
    url = url.strip().replace("http://", "https://")  # Normalize URLs
    match = re.search(r"twitter.com/(@?[A-Za-z0-9_]+)", url)
    return match.group(1).replace("@", "") if match else None

# Set up Selenium WebDriver with Firefox
def setup_driver():
    options = Options()
    options.add_argument("--headless")  # Run in headless mode
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    return webdriver.Firefox(options=options)

# Scrape Twitter profile using Selenium
def scrape_twitter_profile(driver, username):
    if not username:
        return None

    url = f"https://twitter.com/{username}"
    driver.get(url)
    time.sleep(3)  # Shorter wait time

    try:
        WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    except:
        print(f"Page did not load for {username}")
        return None

    def get_element_text(xpath):
        try:
            return driver.find_element(By.XPATH, xpath).text.strip()
        except:
            return "Not Available"

    def get_element_attribute(xpath, attribute):
        try:
            return driver.find_element(By.XPATH, xpath).get_attribute(attribute)
        except:
            return "Not Available"

    return {
        "Username": username,
        "Bio": get_element_text("//div[@data-testid='UserProfileHeader_Items']"),
        "Following Count": get_element_text("//a[contains(@href, '/following')]//span"),
        "Followers Count": get_element_text("//a[contains(@href, '/followers')]//span"),
        "Location": get_element_text("//span[@data-testid='UserLocation']"),
        "Website": get_element_attribute("//a[@data-testid='UserUrl']", "href")
    }

# Step 3: Read Twitter usernames from CSV and scrape data
def scrape_from_csv(input_csv, output_csv):
    df = pd.read_csv(input_csv)
    twitter_profiles = df.iloc[:, 0].dropna().apply(extract_username).dropna().tolist()

    driver = setup_driver()
    scraped_data = []
    for username in twitter_profiles:
        print(f"Scraping: {username}")
        profile_data = scrape_twitter_profile(driver, username)
        if profile_data:
            scraped_data.append(profile_data)
        time.sleep(random.uniform(2, 4))  # Reduced delay

    driver.quit()
    pd.DataFrame(scraped_data).to_csv(output_csv, index=False)
    print(f"Data saved to {output_csv}")

# Main Execution
drive_link = "https://drive.google.com/file/d/1PLYwrGn5YApyWU2QpjbdhM6tea0HuGq7/view?usp=sharing"
input_csv = "twitter_profiles.csv"
output_csv = "scraped_twitter_data.csv"

download_csv(drive_link, input_csv)
scrape_from_csv(input_csv, output_csv)

Downloading...
From: https://drive.google.com/uc?id=1PLYwrGn5YApyWU2QpjbdhM6tea0HuGq7
To: /content/twitter_profiles.csv
100%|██████████| 492/492 [00:00<00:00, 1.28MB/s]


Scraping: whatsapp
Scraping: aacb_CBPTrade
Scraping: aacbdotcom
Scraping: AAWindowPRODUCT
Scraping: aandb_kia
Scraping: ABHomeInc
Scraping: Abrepro
Scraping: ACChristofiLtd
Scraping: aeclothing1
Scraping: AETechnologies1
Scraping: wix
Scraping: AGInsuranceLLC
Data saved to scraped_twitter_data.csv
